#Library Import

In [1]:
import os

In [2]:
os.chdir('C:/Users/Jeric/OneDrive/Desktop/Personal Projects/NYC_restaurants')

In [3]:
import pandas as pd
import numpy as np

In [4]:
nyc = pd.read_csv("C:/Users/Jeric/OneDrive/Desktop/Personal Projects/NYC_restaurants/food_order.csv")

#Preliminary look at data

In [5]:
nyc.head()

,order_id,customer_id,restaurant_name,cuisine_type,cost_of_the_order,day_of_the_week,rating,food_preparation_time,delivery_time
0,1477147,337525,Hangawi,Korean,30.75,Weekend,Not given,25,20
1,1477685,358141,Blue Ribbon Sushi Izakaya,Japanese,12.08,Weekend,Not given,25,23
2,1477070,66393,Cafe Habana,Mexican,12.23,Weekday,5,23,28
3,1477334,106968,Blue Ribbon Fried Chicken,American,29.20,Weekend,3,25,15
4,1478249,76942,Dirty Bird to Go,American,11.59,Weekday,4,25,24


#Find data dimensions (rows and columns)

#find summary statistics

In [6]:
print(nyc.loc[:,~nyc.columns.isin(['order_id','customer_id'])].describe().round(2))

       cost_of_the_order  food_preparation_time  delivery_time
count            1898.00                1898.00        1898.00
mean               16.50                  27.37          24.16
std                 7.48                   4.63           4.97
min                 4.47                  20.00          15.00
25%                12.08                  23.00          20.00
50%                14.14                  27.00          25.00
75%                22.30                  31.00          28.00
max                35.41                  35.00          33.00


#Table of cuisine types

In [7]:
pd.options.display.max_rows = 20

In [8]:
occur = nyc.groupby(["cuisine_type","restaurant_name"]).size().sort_values(ascending=False)
display(occur)

cuisine_type  restaurant_name          
American      Shake Shack                  219
Japanese      Blue Ribbon Sushi            119
Italian       The Meatball Shop            112
American      Blue Ribbon Fried Chicken     96
Italian       Parm                          68
                                          ... 
Mexican       Cafe de La Esquina             1
Japanese      Hiroko's Place                 1
Mexican       Crema Restaurante              1
Indian        Chote Nawab                    1
American      'wichcraft                     1
Length: 179, dtype: int64